In [2]:
import numpy as np
import math
from multiprocessing.pool import ThreadPool as Pool
from numpy.linalg import inv, pinv

import os
default_n_threads = 15
os.environ['OPENBLAS_NUM_THREADS'] = f"{default_n_threads}"
os.environ['MKL_NUM_THREADS'] = f"{default_n_threads}"
os.environ['OMP_NUM_THREADS'] = f"{default_n_threads}"

In [3]:
def divide_into_blocks(matrix: np.ndarray, *args):
    msize = matrix.shape

    blocks_amount = None if not args else args[0]
    if blocks_amount is None or blocks_amount == 0:
        blocks_amount = msize[1]

    if (blocks_amount > msize[1]):
        raise Exception('Blocks amount can not be greater then amount of columns')

    block_size = msize[1] // blocks_amount + (msize[1] % blocks_amount)

    blocks = {}
    start = 0
    for b in range(blocks_amount + 1):
        end = (start + block_size) - 1
        if block_size == 1:
            end = start + 1
        if end > msize[1] - 1:
            end = msize[1]

        block = matrix[:, start:end]
        if block.size != 0:
          blocks[b] = block
        start = end

    return blocks

In [4]:
def pinvblock(block: np.ndarray, block_id: int, pblocks: dict):
  pinv_block = pinv(block)
  pblocks[block_id] = pinv_block

def pinvblocks(blocks: dict) -> dict:
  pinvblocks = {}
  pool = Pool(len(blocks))
  for block_id, block in blocks.items():
      pool.apply_async(pinvblock, (block, block_id, pinvblocks))
  pool.close()
  pool.join()
  return pinvblocks

In [5]:
def form_row(pblock_id: int, pblock: np.ndarray, blocks: dict, rows: dict):
  row = []
  for i in range(len(blocks)):
    if i == pblock_id:
      result = np.eye(pblock.shape[0])
    else:
      result = np.matmul(pblock, blocks.get(i))
    row.append(result)
  rows[pblock_id] = np.concatenate((row),axis=1)


def build_Pmatrix(blocks: dict, pblocks: dict) -> np.ndarray:
  result_rows = {}
  pool = Pool(len(blocks))
  for id, pblock in pblocks.items():
    pool.apply_async(form_row, (id, pblock, blocks, result_rows))
  pool.close()
  pool.join()

  sorted(result_rows)
  rows = list(result_rows.values())
  return np.concatenate(rows,axis=0)

In [6]:
def binv(matrix: np.ndarray, *args):
  blocks = divide_into_blocks(matrix, *args)
  pblocks = pinvblocks(blocks)
  Pmatrix = build_Pmatrix(blocks, pblocks)
  del blocks
  inverted_Pmatrix = inv(Pmatrix)
  sorted(pblocks)
  pseudoinversed = np.matmul(inverted_Pmatrix, np.concatenate(list(pblocks.values()),axis=0) )
  return pseudoinversed

### Description
*binv(matrix: np.ndarray, \*args)* -  the function which returns the Moore-Penrouse pseudo-inversed matrix for full column rank case using block inversion (the formula requires $A^+A=E$). \
Let $A=[A_1 A_2 \dots A_n]$ - full column rank (block) matrix, then the pseudoinversed matrix can be calculated by 
$$A^+ = [A_1 A_2 \dots A_n]^+=\begin{bmatrix}
        \begin{bmatrix}
            A_1^+ \\
            A_2^+ \\
            \dots \\
            A_n^+ \\
        \end{bmatrix}
        [A_1 A_2 \dots A_n]
    \end{bmatrix}^{-1}      \begin{bmatrix}
            A_1^+ \\
            A_2^+ \\
            \dots \\
            A_n^+ \\
        \end{bmatrix}.$$
        

### Usage
By default *binv(matrix: np.ndarray)* receives only matrix. Then it counts that each column is a different block. In that case, there is no advantage and the time of pseudoinversion is equal to the standard function *pinv*. \
**!** For effective calculation pass the amount of blocks in matrix *binv(matrix: np.ndarray, blocks_amount: int)*. Then each block will be calculated in a different thread. 
### Examples
    - binv(np.random.rand(columns, rows)) 
    - binv(np.random.rand(columns, rows), 10)
**Note.** To satisfy the full column rank condition there should be $columns \geq rows$